# Задание

Реализация downsampling блока нейронной сети UNet.
Вспомните общую архитектуру модели UNet. Первая ее часть, encoder,
состоит из блоков (conv + relu -> conv + relu -> maxpoling). Они постепенно
уменьшают размер feature maps и увеличивают количество слоев. Вам
необходимо реализовать этот блок сети UNet. Проверьте вашу реализацию на
примере случайного тензора, чтобы убедиться, что форма выходного тензора
соответствует ожиданиям.

Блок энкодера в модели UNet состоит из 2-х сверток 3*3 с функцией
активации Relu после каждой из них, за которыми следует max_pooling с
ядром 2*2, уменьшающий изображение в 2 раза, следовательно, нужно
применить stride 2.
Для проверки мы возьмем тензор размером 64*284*284. На выходе должны
получить 128*140*140.

Обращаем внимание, что при применении операции свертки размер
изображения уменьшается на 2 пикселя. А значит тут мы не используем
паддинг.


In [4]:
import torch  # Импорт библиотеки PyTorch — основного инструмента для создания и обучения нейронных сетей
import torch.nn as nn  # Импорт подмодуля nn (нейронные сети), который содержит готовые слои и функции активации


In [5]:
class UNetBlock(nn.Module):  # Определяем блок U-Net, наследуем от nn.Module (базовый класс нейросетей в PyTorch)
    def __init__(self, in_channels, out_channels):
        super(UNetBlock, self).__init__()  # Инициализация базового класса

        # Первый сверточный слой:
        # in_channels — число входных каналов (например, 3 для RGB),
        # out_channels — число выходных каналов (фильтров),
        # kernel_size=3 — размер фильтра 3x3,
        # padding=0 — без добавления пикселей на границе, поэтому размер изображения после свертки уменьшится
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=0)

        # Функция активации ReLU:
        # inplace=True — изменяет данные на месте без создания нового объекта (экономит память)
        self.relu = nn.ReLU(inplace=True)

        # Второй сверточный слой:
        # in_channels и out_channels равны out_channels первого слоя,
        # kernel_size и padding аналогичны первому,
        # каждый сверточный слой — отдельный слой с собственными весами
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=0)

        # Операция максимального объединения (max pooling):
        # kernel_size=2 — берет максимум из каждой области 2x2,
        # stride=2 — сдвигается на 2 пикселя, уменьшая размер изображения вдвое по ширине и высоте
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        # Прямой проход данных через блок

        x = self.conv1(x)  # Проходим через первую свертку
        x = self.relu(x)   # Применяем ReLU — обнуляем отрицательные значения
        x = self.conv2(x)  # Проходим через вторую свертку
        x = self.relu(x)   # Снова ReLU

        pooled = self.pool(x)  # Применяем max pooling, уменьшая размер

        # Возвращаем два значения:
        # x — результат после двух сверток (будет использоваться для пропуска (skip connection) в U-Net),
        # pooled — уменьшенная версия для следующего уровня сети
        return x, pooled


In [6]:
# Пример использования блока UNetBlock с комментариями

block = UNetBlock(64, 126)
# Создаем объект блока, где:
# 64 — число входных каналов (например, число признаков с предыдущего слоя)
# 126 — число выходных каналов (число фильтров, которые блок должен обучить)

input_tensor = torch.randn(1, 64, 284, 284)
# Создаем входной тензор с случайными значениями:
# 1 — размер батча (один пример),
# 64 — число каналов (например, 64 признака),
# 284x284 — высота и ширина изображения (пиксели)

conv_output, pooled_output = block(input_tensor)
# Передаем input_tensor в блок:
# conv_output — результат после двух сверток (без уменьшения размерности пулингом),
# pooled_output — результат после max pooling (с уменьшенной размерностью)

print(conv_output.shape, pooled_output.shape)
# Выводим размеры тензоров:
# conv_output.shape — размер после двух сверток (будет меньше из-за отсутствия паддинга),
# pooled_output.shape — размер после уменьшения max pooling'ом (в 2 раза меньше по ширине и высоте)

# Почему размеры такие:
# Каждая свертка с kernel_size=3 и padding=0 уменьшает размер на 2 пикселя с каждой стороны,
# Значит 2 свертки уменьшат размер с 284 до 280 (284 - 2*2 = 280)
# После этого max pooling с kernel_size=2 и stride=2 уменьшит 280 до 140 (половина)

# Ожидаемые размеры:
# conv_output.shape -> (1, 126, 280, 280)
# pooled_output.shape -> (1, 126, 140, 140)


torch.Size([1, 126, 280, 280]) torch.Size([1, 126, 140, 140])
